# Ultimate Cheat Sheet
A compilation of code snippets for a mix of use cases, tools or languages with a loose organization. 

Search index for categories. 

---

## Index

- [Bash](#bash-scripts)
- [Git](#git)
- [DBT](#dbt)
- [Terraform](#terraform)
- [Python](#python)

## Bash Scripts

#### Rename all files in a directory

Rename all files in a dir substituting '.'->dot, with ' '->empty space, and removing everything aftert the 4th dot. Usefull for renaming bulk downloads with a structured naming  
Attention: edge case names might cause issues

```bash
preview=true # To actually rename, set preview=false
for f in *.*; do  # Loop through all files with at least one dot in the name
    # Use awk to split filename by '.' and keep only up to the 4th part  
    newname=$(echo "$f" | awk -F'.' '{ 
        new=""; 
        for (i=1; i<=NF && i<=4; i++) new = new $i " ";  
        print new 
    }' | sed 's/ *$//') # Replace dots with spaces, remove trailing spaces  

    # Check if the new filename already exists to avoid overwriting  
    if [ -e "$newname" ]; then
        echo "Skipping '$f': already exists."
        continue
    fi
    # Preview mode: show the mv command instead of executing it  
    if ["$preview" = true]; then
        echo mv -- "$f" "$newname" 
        break # Break after first preview to avoid too much output in preview mode
    else
        mv -- "$f" "$newname"
    fi
done

## Git
Using git always be as specific and explicit as possible

#### Clone a repo

To clone a github repository there a few different ways from downloading a zip to using an ssh key saved in the github account  
Example using a ssh link
```bash
git clone git@github.com:company/RepoName.git #paste in the ssh link from the github repository page
```

#### Provide a user id

When doing operations like push or merge github needs to know who is doing these operations, id yourself using:
```bash
git config --global user.email "my.email@company.com"
git config --global user.name "Goncalo Felicio"
```

#### Make a branch

When working with multiple people its better to ALLWAYS work in separate branches, so first thing is creating a branch
This creates and moves you to the new branch
```bash
git checkout -b my-branch-name
```

#### Discard Changes

If you mess up and your local repo has problems or you simply want to discard changes from conflicts, can reset the local repo to the current remote one:
```bash
git reset --hard origin/master
```

#### Check Previous Commits

To see previous commits info and catch up can:
```bash
git log
```

#### Add and Commit changes 

Installing pre-commit and setting up a pre-commit yaml file lets us run tests and hooks before committing changes 
```bash
pre-commit install
pre-commit run -a
```
After installing pre commits run automatically

Stage all changes with "." then give the commit the message in "something short but descriptive, can also include issue number

```bash
git add .
git commit -m "modified file for issue #123 "
```

Can also create fixup commits that disappear after a merge or rebase to keep history line cleaner, this is for super small fixes

```bash
git commit --fixup head
```

#### Push to remote branch (NOT MASTER)

push and set my local branch to track the specified remote branch
```bash
git push --set-upstream origin branch-name
```

Fixup commits, be careful with referencing fixup commits to eachother or to commits that no longer exist
```bash
git push --set-upstream origin branch-name
```

#### Create a pull request
After pushing the branch can go to github and start a pull request, adding all the necessary info and reviewers no github

#### Fix Conflicts and Push again
If there are conflicts in the pull request, cna edit my file to fix it, then push again

```bash
git push --set-upstream origin branch-name
```

#### Rebase a branch
Rebasing or merging to master are very similar, but rebasing is better if a master has many many branches, by keeping the history linear -> each time a master is rebased, while merging keeps the branch timeline intact better for single developer, rebasing moves it to the present


```bash
git rebase master    
git rebase --continue

git merge master
```

## DBT
DBT is a data build tool to build pipelines from raw data to user friendly marts in modular, reusable SQL queries, with easy version control, testing and documentation

The workflow is usually:
- create a raw table in database to use as source
- create a stg model to pull that data:
    - include renaming cols, casting data types, basic cleaning, filter irrelevant data, as views these are what int models reference
*stg_raw_schema__raw_data.sql*
```sql
with raw_data as (
    select *
    from {{ source('raw_schema'.'raw_table') }}
)
select * 
from raw_data
```

- modify stg_models.yml and stg_sources.yml to include this new model and source
- create an intermediate model to trasform the data:
    - include joins, more complex calcs and aggs, reusability/modularity, as views these are the "CTEs" the marts reference from
*int_schema_data.sql*
```sql
with 
int_data as(
    select a,b,c 
    from {{ ref('stg_raw_schema__raw_data')}}
),
int_data_2 as(
    select d,e,f 
    from {{ ref('stg_raw_schema__raw_data2')}}
)
select *
from int_data as i1
join int_data_2 as i1 on i1.a=i2.d
```

- modify int_models.yml to include this new model
- create mart models for user access:
    - include user-friendly naming, wide denormalized tables, aggregations, materialized, always reference int models
*mart_schema_data.sql*
```sql
with 
business_data as(
    select a,b,c 
    from {{ ref('int_schema_data')}}
)
select *
from business_data 
```

#### Run DBT 
in a targeted way to avoid building all models again


```bash
db2 # to initialize dbt

dbt run --select +top_model_name+
```

This will take the model named and all dependencies and build them

## Terraform

terraform is a versioned configuration manager for every resource, letting us manage all the infrastructure, jobs, vms, etc safely in one spot, becase terraform is versioned it only applies after passing the CI/CD and merging to the master repo

to check what differences my local has to the current state
```tf
terraform plan
```
to apply my local changes to the current state - WARNING - extremely careful not to delete current infrastructure
```tf
terraform apply

terraform apply -target=<RESOURCE_TYPE>.<RESOURCE_NAME>
```
for targeting a specific resource and not all the config

## Python

Python snippets for common libraries and use cases